In [2]:
import pandas as pd 
import numpy as np

In [3]:
dataset=pd.read_csv(r'C:\Users\vs390\OneDrive\Desktop\Swiggy instamart delivery prediction\data\test.csv')
dataset['Date']=pd.to_datetime(dataset['Date'],errors='coerce')
dataset['Day_of_Week']=dataset['Date'].dt.day_name()
dataset.head(2)

,DateTime,Date,Hour,Day_of_Week,Location,Product,Order_Volume,Weather,Traffic,Is_Holiday,...,Distance_to_Warehouse,Customer_Age_Group,Feedback_Score,Item_Returned,Discount_Amount,Gift_Wrapping_Requested,Time_Since_Last_Order,Weather_Severity,Traffic_Congestion_Index,Warehouse_Proximity_Score
0,2023-01-01 00:00:00,2023-01-01,0,Sunday,Koramangala,Milk,143,Clear,Moderate,0,...,19.320305,26-35,4.089132,True,80.454587,True,23.395002,1.366587,1.985490,7.916775
1,2023-01-01 00:00:00,2023-01-01,0,Sunday,Koramangala,Bread,121,Clear,Moderate,0,...,16.529100,18-25,4.403027,False,43.380880,True,20.877744,1.421246,1.043545,5.236938


In [4]:
product_data=dataset[(dataset['Product']=='Milk') &
                     (dataset['Order_Volume'])]

In [14]:
output=[]
window_size=7
for location in product_data['Location'].unique():
    loc_data=product_data[product_data['Location']==location]
    
    for i in range(len(loc_data)):
        if i < window_size :
            actual_value=loc_data['Order_Volume'].iloc[i]
            predicted_volume=None 
            percentage_change=None
            output.append([location,i+1,actual_value,None,predicted_volume,percentage_change])
        else:
            alpha=0.6
            ewma_pred=loc_data['Order_Volume'].iloc[i - window_size:i].ewm(span=window_size, adjust=False).mean().iloc[-1]
            actual_value=loc_data['Order_Volume'].iloc[i]   
            percentage_change=((actual_value-ewma_pred)/ewma_pred)*100 if ewma_pred !=0 else 0
            output.append([location,i+1,actual_value,i+1,ewma_pred,percentage_change])
output_df=pd.DataFrame(output,columns=['Location','Day','Order_Volume','Predicted_day','Predicted_Volume','Percentage_Change'])
output_df.head(14)

,Location,Day,Order_Volume,Predicted_day,Predicted_Volume,Percentage_Change
0,Koramangala,1,143,NaN,NaN,NaN
1,Koramangala,2,145,NaN,NaN,NaN
2,Koramangala,3,123,NaN,NaN,NaN
3,Koramangala,4,121,NaN,NaN,NaN
4,Koramangala,5,121,NaN,NaN,NaN
5,Koramangala,6,459,NaN,NaN,NaN
6,Koramangala,7,141,NaN,NaN,NaN
7,Koramangala,8,117,8.0,194.872559,-39.960762
8,Koramangala,9,139,9.0,175.671387,-20.874991
9,Koramangala,10,175,10.0,163.566895,6.989865
